# 🔧 LangGraph Graph API 완전 정복 가이드

## 📚 개요

**LangGraph Graph API**는 마치 **모듈형 컴포넌트를 조립하듯이** 복잡한 AI 시스템을 만들 수 있게 해주는 강력한 도구입니다! 🧩

### 🏗️ Graph API가 특별한 이유

일반적인 프로그래밍에서는 코드가 위에서 아래로 순서대로 실행됩니다. 하지만 Graph API는 다릅니다:

- **🔀 유연한 흐름**: 상황에 따라 다른 경로로 실행
- **🤖 AI 에이전트 협력**: 여러 AI가 함께 문제 해결
- **🔄 반복과 루프**: 만족할 때까지 재시도
- **🎯 조건부 실행**: 똑똑한 의사결정

### 🎯 이 튜토리얼에서 배울 것들

1. **🌐 그래프 기본 개념** - State, Nodes, Edges가 뭔지 쉽게 이해하기
2. **📊 상태 관리** - 데이터를 어떻게 저장하고 관리하는지
3. **🔨 노드 심화 학습** - 실제 작업을 수행하는 함수들
4. **🔀 엣지와 라우팅** - 다음에 어디로 갈지 결정하는 방법
5. **🚀 고급 기능** - Send, Command 등 전문가 기능들

### 🎮 실생활 비유로 이해하기

**LangGraph**를 **회사 업무 시스템**에 비유해보세요:

```
🏢 회사 (Graph)
├── 📋 업무 현황판 (State) - 모든 정보가 기록되는 곳
├── 👥 직원들 (Nodes) - 실제 일을 처리하는 사람들  
└── 📞 소통 규칙 (Edges) - 누구에게 언제 일을 넘길지 정하는 규칙
```

### 💡 핵심 철학

> **"각자의 역할이 명확하고, 소통이 원활한 팀이 최고의 결과를 만든다"**
> 
> _노드는 작업을 수행하고, 엣지는 다음 단계를 결정한다_ 🎯

### 🚀 준비되셨나요?

이제 실제 예제를 통해 LangGraph의 강력한 세계로 들어가봅시다! 


## 환경 설정 🛠️

LangGraph 기능을 체험하기 전에 필요한 도구들을 준비해봅시다!

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangGraph-Tutorial")

---

# Part 1: 그래프 기본 개념 🌐

## 1.1 그래프가 뭐길래? 🤔

LangGraph에서 말하는 **"그래프"**는 수학 그래프가 아니라 **"업무 흐름도"**라고 생각하시면 됩니다! 

### 🏭 공장 생산라인으로 이해하기

공장에서 제품을 만드는 과정을 떠올려보세요:

```
📦 원재료 → 🔨 가공1 → ⚙️ 가공2 → 📋 검수 → ✅ 완제품
```

LangGraph도 이와 똑같습니다:

- **📦 원재료** = 입력 데이터 (사용자 질문, 텍스트 등)
- **🔨 가공 과정** = 노드들 (AI 모델, 함수들)
- **📋 중간 결과** = 상태 (State)
- **✅ 완제품** = 최종 결과

### 🎯 그래프의 3가지 핵심 구성요소

#### 1️⃣ **State (상태) - 📋 작업 현황판**
- **실생활 비유**: 공장의 **생산 현황판**
- **역할**: 현재 진행 상황과 데이터를 모두 기록
- **예시**: 현재 처리 중인 메시지, 단계별 결과들

#### 2️⃣ **Nodes (노드) - 👷 작업자들**  
- **실생활 비유**: 각 **작업 스테이션의 직원들**
- **역할**: 실제 작업을 수행하는 함수들
- **예시**: AI 모델 호출, 데이터 가공, 결과 검증

#### 3️⃣ **Edges (엣지) - 🚛 운송 경로**
- **실생활 비유**: **컨베이어 벨트와 운송 경로**
- **역할**: 다음에 어느 작업자에게 보낼지 결정
- **예시**: "A 작업 완료 → B 작업으로 이동"

### 🔄 LangGraph의 똑똑한 실행 방식

일반적인 프로그램과 달리 LangGraph는 **Google의 Pregel 시스템**에서 영감을 받았습니다:

1. **⚡ Super-step**: 한 번에 하나의 작업 단계 실행
2. **🔀 병렬 처리**: 같은 단계의 여러 작업을 동시에 실행
3. **📝 순차 기록**: 각 단계의 결과를 차근차근 기록
4. **🎯 스마트 종료**: 모든 작업이 끝나면 자동으로 멈춤

### 💡 왜 이런 방식이 좋을까?

✅ **유연성**: 상황에 따라 다른 경로로 처리  
✅ **확장성**: 새로운 작업자(노드) 추가 용이  
✅ **디버깅**: 각 단계별 상태 추적 가능  
✅ **재사용**: 만들어진 그래프를 다른 곳에서도 활용

이제 실제 코드로 간단한 그래프를 만들어봅시다! 🚀

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict


# 간단한 그래프 예제
class SimpleState(TypedDict):
    """Simple state with a single field"""

    message: str


# StateGraph 생성
graph_builder = StateGraph(SimpleState)


# 노드 함수 정의
def processor(state: SimpleState):
    """Process the message in the state"""
    # 노드는 현재 상태를 입력으로 받고 업데이트를 반환
    return {"message": state["message"] + " - processed"}


# 노드 추가
graph_builder.add_node("processor", processor)

# 엣지 추가 (진입점과 종료점)
graph_builder.add_edge(START, "processor")
graph_builder.add_edge("processor", END)

# 그래프 컴파일 - 필수!
graph = graph_builder.compile()

In [ ]:
from langchain_teddynote.graphs import visualize_graph

visualize_graph(graph)

In [ ]:
# 그래프 실행
result = graph.invoke({"message": "Hello LangGraph"})
print(f"결과: {result}")

### 1.2 그래프 컴파일 - 🏭 공장 가동 준비!

그래프를 만든 후에는 **반드시 컴파일**해야 합니다! 이는 마치 **공장 가동 전 최종 점검**과 같습니다.

#### 🔍 컴파일 과정에서 일어나는 일들

1. **🛠️ 구조 검증**: "모든 작업자가 제자리에 있나?"
   - 연결되지 않은 노드(고아 노드) 확인
   - 실행 경로가 제대로 이어져 있는지 검증

2. **⚙️ 설정 적용**: "특별한 옵션이 필요한가?"
   - 체크포인터: 중간 결과 저장소 설정
   - 브레이크포인트: 디버깅을 위한 중단점

3. **🚀 실행 준비**: "이제 언제든 가동할 수 있어!"
   - 최적화된 실행 가능한 객체 생성

#### 💡 컴파일 옵션들

- **💾 checkpointer**: 그래프 실행 중간에 상태를 저장 (메모리 기능!)
- **⏸️ interrupt_before**: 특정 노드 실행 **전**에 일시정지
- **⏹️ interrupt_after**: 특정 노드 실행 **후**에 일시정지

이 옵션들은 복잡한 시스템을 디버깅하거나 사용자 확인이 필요할 때 매우 유용합니다!

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

# 체크포인터와 함께 컴파일
memory = InMemorySaver()

graph_with_memory = graph_builder.compile(
    checkpointer=memory,  # 상태 저장을 위한 체크포인터
    # interrupt_before=["processor"],  # 특정 노드 전에 중단 (디버깅용)
    # interrupt_after=["processor"],   # 특정 노드 후에 중단
)

print("✅ 체크포인터가 있는 그래프 컴파일 완료!")

---

# Part 2: State (상태) 심화 학습 📊

## 2.1 State가 뭐고 왜 중요할까? 🤔

**State(상태)**는 그래프의 **기억 저장소**입니다! 마치 **데이터베이스의 백업 파일**처럼 현재 진행 상황을 모두 기록해두는 곳이죠.

### 🏨 호텔 프런트 데스크로 이해하기

호텔 프런트 데스크를 생각해보세요:

- **📋 투숙객 대장** = State 스키마 (어떤 정보를 기록할지 정의)
- **✍️ 기록 방식** = 리듀서 함수 (정보를 어떻게 업데이트할지)
- **📝 실제 기록** = 현재 상태 값들

### 🎯 State 스키마 정의 방법 3가지

LangGraph에서는 State를 정의하는 세 가지 방법이 있습니다:

#### 1️⃣ **TypedDict 방식** (⭐ 가장 추천!)
```python
class MyState(TypedDict):
    name: str
    count: int
```
- **장점**: 가벼움, 빠름, 간단함
- **단점**: 기본값 설정이 번거로움
- **언제 사용**: 대부분의 경우

#### 2️⃣ **Dataclass 방식**
```python
@dataclass
class MyState:
    name: str = "기본값"
    count: int = 0
```
- **장점**: 기본값 설정 쉬움
- **단점**: TypedDict보다 약간 무거움
- **언제 사용**: 기본값이 많이 필요할 때

#### 3️⃣ **Pydantic 방식**
```python
class MyState(BaseModel):
    name: str = "기본값"
    count: int = 0
```
- **장점**: 강력한 유효성 검증
- **단점**: 가장 무거움, 복잡함
- **언제 사용**: 엄격한 데이터 검증이 필요할 때

### 💡 어떤 방식을 선택해야 할까?

**입문자라면 TypedDict**부터 시작하세요! 간단하고 직관적이며 성능도 우수합니다. 🚀

In [ ]:
from typing import Annotated, List
from dataclasses import dataclass, field
from pydantic import BaseModel


# 방법 1: TypedDict (가장 일반적, 성능 우수)
class TypedDictState(TypedDict):
    """State using TypedDict"""

    count: int
    items: List[str]


# 방법 2: dataclass (기본값 지원)
@dataclass
class DataclassState:
    """State using dataclass with default values"""

    count: int = 0
    items: List[str] = field(default_factory=list)


# 방법 3: Pydantic (재귀적 검증, 성능은 떨어짐)
class PydanticState(BaseModel):
    """State using Pydantic for validation"""

    count: int = 0
    items: List[str] = []


print("✅ 세 가지 방식의 State 스키마 정의 완료!")

## 2.2 Reducers (리듀서) - 📝 업데이트 규칙 관리자

**리듀서**는 "새로운 정보가 들어왔을 때 기존 정보와 어떻게 합칠까?"를 정하는 **똑똑한 규칙**입니다!

### 🏪 상점 재고 관리로 이해하기

상점에서 재고를 관리한다고 생각해보세요:

#### 📦 **기본 리듀서 (덮어쓰기)**
```python
재고_수량: int  # 새 값으로 완전히 교체
```
**예시**: "사과 10개" → "사과 15개" (10개는 사라지고 15개가 됨)

#### 📋 **add 리듀서 (추가)**
```python
주문_목록: Annotated[List[str], add]  # 기존 목록에 추가
```
**예시**: ["사과", "바나나"] + ["오렌지"] = ["사과", "바나나", "오렌지"]

#### 🏆 **커스텀 리듀서 (최대값 유지)**
```python
최고_판매량: Annotated[int, lambda old, new: max(old, new)]
```
**예시**: 기존 50개, 새로 30개 → 50개 유지 (더 큰 값 선택)

### 🎯 리듀서의 핵심 원리

1. **🔄 자동 병합**: 노드에서 업데이트할 때 자동으로 적절히 합쳐짐
2. **🛡️ 데이터 보호**: 실수로 중요한 데이터를 잃어버리는 것 방지
3. **🎨 유연성**: 상황에 맞는 업데이트 전략 선택 가능

### 💡 언제 어떤 리듀서를 쓸까?

- **기본 리듀서**: 설정값, 현재 상태 등 (예: 현재_온도)
- **add 리듀서**: 메시지, 로그, 히스토리 등 (예: 대화_내역)  
- **커스텀 리듀서**: 특별한 로직이 필요할 때 (예: 최고_점수)

다음 예제에서 리듀서들이 실제로 어떻게 동작하는지 확인해봅시다! 🚀

In [ ]:
from operator import add


# 기본 리듀서 vs 커스텀 리듀서
class StateWithReducers(TypedDict):
    """State with custom reducers"""

    # 기본 리듀서: 값을 덮어씀
    current_value: int

    # add 리듀서: 리스트를 연결
    history: Annotated[List[str], add]

    # 커스텀 리듀서: 최대값 유지
    max_value: Annotated[int, lambda x, y: max(x, y)]


# 그래프 생성
builder = StateGraph(StateWithReducers)


def update_values(state: StateWithReducers):
    """Update various state values"""
    return {
        "current_value": 10,  # 덮어쓰기
        "history": ["새 항목"],  # 추가
        "max_value": 15,  # 최대값과 비교
    }


builder.add_node("updater", update_values)
builder.add_edge(START, "updater")
builder.add_edge("updater", END)

graph = builder.compile()

# 그래프 시각화
visualize_graph(graph)

In [ ]:
# 초기 상태로 실행
initial_state = {"current_value": 5, "history": ["초기 항목"], "max_value": 20}

result = graph.invoke(initial_state)
print("\n리듀서 적용 결과:")
print(f"  current_value: 5 → {result['current_value']} (덮어쓰기)")
print(f"  history: {result['history']} (추가)")
print(f"  max_value: {result['max_value']} (최대값 유지)")

## 2.3 메시지 처리 - 💬 대화의 기억을 남기기

**대화형 AI**에서는 이전 대화 내용을 기억하는 것이 정말 중요하죠! LangGraph는 이를 위한 **특별한 메시지 관리 시스템**을 제공합니다.

### 📱 카카오톡 대화방으로 이해하기

카카오톡 대화방을 떠올려보세요:

- **💬 대화 내역**: 지금까지 주고받은 모든 메시지들
- **➕ 새 메시지**: 기존 대화에 추가되는 새로운 메시지  
- **📝 메시지 타입**: 텍스트, 이미지, 파일 등 다양한 형태

### 🎯 LangGraph의 메시지 시스템

#### 방법 1: `add_messages` 리듀서 사용 (직접 정의)
```python
class ChatState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    metadata: dict  # 부가 정보
```

#### 방법 2: `MessagesState` 상속 (⭐ 더 간단!)
```python
class ExtendedChatState(MessagesState):
    # messages는 이미 정의되어 있음!
    metadata: dict  # 필요한 것만 추가
```

### 🔍 메시지 타입들

- **🙋 HumanMessage**: 사용자가 보낸 메시지
- **🤖 AIMessage**: AI가 답변한 메시지  
- **🛠️ SystemMessage**: 시스템 지시사항
- **⚙️ ToolMessage**: 도구 실행 결과

### 💡 왜 이렇게 관리할까?

✅ **대화 맥락 유지**: 이전 대화를 참고해서 더 나은 답변  
✅ **자동 추가**: 새 메시지가 알아서 대화 목록에 추가됨  
✅ **타입 안전성**: 메시지 종류별로 적절한 처리  
✅ **디버깅 용이**: 대화 과정을 단계별로 추적 가능

실제 예제를 통해 메시지가 어떻게 관리되는지 확인해봅시다! 💬

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage
from langgraph.graph.message import add_messages
from langgraph.graph import MessagesState


# 방법 1: add_messages 리듀서 사용
class ChatState(TypedDict):
    """State with message handling"""

    messages: Annotated[List[AnyMessage], add_messages]
    metadata: dict


# 방법 2: MessagesState 상속 (더 간단)
class ExtendedChatState(MessagesState):
    """Extended state inheriting from MessagesState"""

    # messages 키는 이미 정의되어 있음
    metadata: dict

In [ ]:
from langchain_core.runnables import RunnableConfig

# 메시지 처리 예제
builder = StateGraph(ChatState)


def process_chat(state: ChatState):
    """Process chat messages"""
    # 마지막 메시지 접근
    last_message = state["messages"][-1]

    # 응답 생성
    response = AIMessage(content=f"{last_message.content}")

    return {
        "messages": [response],  # add_messages가 자동으로 추가
        "metadata": {"processed": True},
    }


builder.add_node("chat", process_chat)
builder.add_edge(START, "chat")
builder.add_edge("chat", END)

memory = InMemorySaver()

chat_graph = builder.compile(checkpointer=memory)

In [ ]:
config = RunnableConfig(configurable={"thread_id": "1"})

# 실행
result = chat_graph.invoke(
    {
        "messages": [HumanMessage(content="안녕하세요")],
        "metadata": {},
        "user_info": {"name": "사용자"},
    },
    config=config,
)

print("\n메시지 처리 결과:")
for msg in result["messages"]:
    print(f"  [{msg.__class__.__name__}]: {msg.content}")

## 2.4 다중 스키마 - 🎭 역할에 따른 다른 모습

때로는 **같은 데이터라도 보는 관점에 따라 다르게 표현**해야 할 때가 있습니다. 이럴 때 다중 스키마가 빛을 발합니다!

### 🏪 온라인 쇼핑몰로 이해하기

온라인 쇼핑몰에서 하나의 주문을 생각해보세요:

#### 👤 **고객이 보는 화면** (InputState)
```
주문번호: ORD-2024-001
상품: 스마트폰 갤럭시 S24
수량: 1개
```

#### 📦 **창고에서 보는 화면** (InternalState)  
```
SKU: PHONE_SAMSUNG_S24_128GB_BLACK
재고위치: A-3-15
픽업담당자: 김창고
```

#### 📊 **관리자가 보는 화면** (OutputState)
```
처리상태: 배송준비완료
예상배송일: 2024-01-15
고객만족도: 92%
```

### 🎯 다중 스키마의 핵심 개념

#### **InputState** - 🚪 입구에서 받는 정보
- 그래프가 시작할 때 **사용자가 제공하는 데이터**의 형태
- **예시**: 사용자 질문, 업로드한 파일, 설정값 등

#### **InternalState** - 🏭 내부에서 사용하는 정보  
- 그래프 **내부 노드들이 실제로 사용하는** 데이터 구조
- **예시**: 중간 처리 결과, 임시 데이터, 내부 상태값 등

#### **OutputState** - 🎁 최종 결과물
- 그래프가 **완료되었을 때 사용자에게 반환하는** 데이터 형태
- **예시**: 최종 답변, 처리 결과, 요약 정보 등

### 💡 왜 이렇게 복잡하게 나눌까?

✅ **보안**: 내부 처리 로직을 사용자에게 노출하지 않음  
✅ **단순화**: 사용자는 필요한 정보만 간단하게 제공  
✅ **유연성**: 내부 구조 변경해도 외부 인터페이스는 그대로  
✅ **명확성**: 입력/출력/내부 처리가 명확히 구분됨

실제 예제에서 이 세 가지 스키마가 어떻게 협력하는지 살펴봅시다! 🚀

In [ ]:
# 그래프의 입력 스키마
class InputState(TypedDict):
    """Input schema - what users provide"""

    user_input: str


# 그래프의 출력 스키마
class OutputState(TypedDict):
    """Output schema - what users receive"""

    graph_output: str


# 내부에서 사용되는 State
class InternalState(TypedDict):
    """Internal schema - full state for processing"""

    user_input: str
    internal_state: str


# Private State
class PrivateState(TypedDict):
    """Private schema - for internal node communication"""

    private_state: str

In [ ]:
# 다중 스키마를 사용하는 그래프
builder = StateGraph(
    InternalState,
    input_schema=InputState,  # 입력 스키마 지정
    output_schema=OutputState,  # 출력 스키마 지정
)


def node_1(state: InputState) -> InternalState:
    """First node - reads from InputState, writes to InternalState"""
    # InputState를 받지만 InternalState의 키에 쓸 수 있음
    print(f"node_1: {state}")
    return {
        "user_input": state["user_input"],
        "internal_state": state["user_input"] + "(node_1 에서 처리됨)",
    }


def node_2(state: InternalState) -> PrivateState:
    """Second node - reads from InternalState, writes to PrivateState"""
    print(f"node_2: {state}")
    # 새로운 state 채널(temp_data) 추가 가능
    return {"private_state": f"PRIVATE: {state['internal_state']} (node_2 에서 처리됨)"}


def node_3(state: PrivateState) -> OutputState:
    """Final node - reads from PrivateState, writes to OutputState"""
    print(f"node_3: {state}")
    return {"graph_output": f"최종 결과: {state['private_state']} (node_3 에서 처리됨)"}


# 노드와 엣지 추가
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
builder.add_edge("node_3", END)

multi_schema_graph = builder.compile()

visualize_graph(multi_schema_graph)

# 실행 - 입력은 InputState 형식
result = multi_schema_graph.invoke({"user_input": "사용자 입력"})

# 출력은 OutputState 형식
print(f"\n다중 스키마 결과: {result}")